# Bab 9 Extreme Learning Machine

## Praktikum

### a) Fungsi *Training* ELM

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
import time
import numpy as np

def elm_fit(X, target, h, W=None):
  start_time = time.time()

  if W is None:
    W = np.random.uniform(-.1, .1, (h, len(X[0])))

  Hinit = X @ W.T
  H = 1 / (1 + np.exp(-Hinit))
  Ht = H.T
  Hp = np.linalg.inv(Ht @ H) @ Ht
  beta = Hp @ target
  y = H @ beta
  mape = sum(abs(y - target) / target) * 100 / len(target)

  execution = time.time() - start_time
  print("Waktu eksekusi: %s detik" % execution)

  return W, beta, mape

### b) Fungsi *Testing* ELM

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
def elm_predict(X, W, b, round_output=False):
  Hinit = X @ W.T
  H = 1 / (1 + np.exp(-Hinit))
  y = H @ b

  if round_output:
    y = [int(round(x)) for x in y]

  return y

### c) Klasifikasi *Dataset* Iris

![Iris Dataset](https://www.spataru.at/images/blog/iris-dataset-svm/iris_types.jpg)

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = iris.target
Y += 1

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)
W, b, mape = elm_fit(X_train, y_train, 3)

print('MAPE:', mape)

output = elm_predict(X_test, W, b, round_output=True)
accuracy = accuracy_score(output, y_test)

print('Output:', output)
print('True :', y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.00036787986755371094 detik
MAPE: 9.625621883226
Output: [1, 3, 3, 1, 2, 3, 3, 3, 1, 1, 3, 2, 2, 3, 2, 2, 2, 2, 1, 1, 1, 3, 1, 3, 3, 1, 3, 2, 3, 3, 3, 2, 2, 2, 1, 3, 3, 2, 1, 3, 2, 1, 3, 3, 2]
True : [1 3 3 1 2 3 3 3 1 1 3 2 3 3 2 3 2 2 1 1 1 3 1 3 2 1 3 2 3 3 3 2 2 2 1 3 3
 2 1 3 2 1 3 3 2]
Accuracy: 0.9333333333333333


# Analisis

### d) Fungsi *Training* Backpropagation

In [ ]:
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    np.random.seed(1)
    nin = [np.empty(i) for i in layer_conf]
    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)]
    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n)-1):
        n[i][-1] = 1

    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0

        for r in range(len(X)):
            n[0][:-1] = X[r]

            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1])
                n[L][:len(nin[L])] = sig(nin[L])

            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))

            for L in range(len(layer_conf) - 1, 1, -1):
                din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
                d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
                dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))

            # Update weights
            for i in range(len(w)):
                w[i] += dw[i]

        mse /= len(X)

        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')

    return w, epoch, mse

### e) Fungsi *Testing* Backpropagation

In [ ]:
def bp_predict(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []

  n.append(np.empty(len(w[-1][0])))

  for x in X:
    n[0][:-1] = x

    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = sig(nin[L])

    predict.append(n[-1].copy())

  return predict

### f) Klasifikasi *Dataset* Iris

In [ ]:
# Fungsi sigmoid dan turunannya
def sig(x):
    return 1 / (1 + np.exp(-x))

def sigd(x):
    return sig(x) * (1 - sig(x))

# Fungsi one-hot encoding dan decoding
def onehot_enc(target):
    return np.eye(len(np.unique(target)))[target]

def onehot_dec(encoded):
    return np.argmax(encoded, axis=1)

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 3, 3), learn_rate=0.1, max_epoch=100, max_error=0.5, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epochs: 23, MSE: 0.4858416685844551
Output: [0 2 2 0 2 2 2 0 0 2 2 0 2 2 2 0 2 2 0 0 2 2 2 0 2 2 0 0 2 2 2 2 2 2 2 0 2
 0 2 2 2 0 2 2 2]
True : [0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]
Accuracy: 0.6
